In [1]:
#Data Exploration
import numpy as np
import pandas as pd
import xgboost ; import lightgbm

ImportError: No module named 'xgboost'

In [2]:
#read in data
train = pd.read_csv("train.csv")
members = pd.read_csv('members.csv')
songs = pd.read_csv('songs.csv')
song_extra_info = pd.read_csv('song_extra_info.csv')

In [12]:
rowin = train['msno'].nunique()
colin = train['song_id'].nunique()
train.shape[0] /(rowin * colin) #really really sparse data

In [152]:
#occurence preparation
trainme = train.merge(songs,on = 'song_id',how = 'left')[['msno','genre_ids','target']]
trainme = trainme[~pd.isnull(trainme.genre_ids)]
v = trainme.groupby(['msno','genre_ids','target']).size()
v = v.reset_index()
v.columns = ['msno','genre_ids','target','count']

In [99]:
# song_length become second or minute + seconds
round(songs.song_length.head()/1000%60)
#genres
from collections import defaultdict
song_gen = defaultdict(int)
for ids in songs.genre_ids:
    if not pd.isnull(ids):
        for sub in ids.split("|"):
            song_gen[sub] += 1

In [258]:
#compute the occurrence matrix based on genres
songmix = list(set(list(song_gen.keys()) + list(songs.genre_ids.unique())))
songmix = [(i,j) for i in songmixe for j in range(2)]
dicti = dict(zip(songmix,range(len(songmix))))
v['genre_ids_map'] = v.apply(lambda row: dicti[(row['genre_ids'],row['target'])],axis = 1)
dictm = dict(zip(trainme.msno.unique(),range(trainme.msno.nunique())))
v['msno_ids_map'] = v['msno'].apply(lambda x: dictm[x])
#cooccurence matrix 
mat = np.zeros((v.msno.nunique(),len(songmix)))
for i in range(v.shape[0]):
    mat[v.msno_ids_map[i],v.genre_ids_map[i]] = v['count'][i]
colindex = []
for i,j in dicti.items(): #iteritems for python 3
    if not pd.isnull(i[0]) and "|" in i[0]:
        colindex.append(j)
        for ind in i[0].split("|"):
            mat[:,dicti[(ind,i[1])]] += mat[:,j]
dropindex = []
for i, j in dicti.items():
    if not pd.isnull(i[0]) and "|" in i[0]:
        dropindex.append(j)
        for sub in i[0].split("|"):
            mat[:,dicti[(sub,i[1])]] += mat[:,j]
coln = [str(songmix[i][0])+str('_')+ str(songmix[i][1]) for i in range(len(songmix))]
mat = pd.DataFrame(mat,index = trainme.msno.unique(),columns = songmix)
mat = mat.iloc[:,[i for i in range(len(songmix)) if i not in colindex]]
mat = mat.loc[:,matt.sum(axis = 0) != 0]

In [4]:
#How many days has they been active
members['rtime'] = members['registration_init_time'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['etime'] = members['expiration_date'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['numactivedays'] = (members['etime'] - members['rtime']).dt.days
min_day = members['rtime'].min()
members['day_id_res'] = (members['rtime'] - min_day).dt.days+1
members['day_id_exp'] = (members['etime'] - min_day).dt.days+1
members['r_year'] = members['rtime'].dt.year
members['r_month'] = members['rtime'].dt.month
members['r_day'] = members['rtime'].dt.day
members['e_year'] = members['etime'].dt.year
members['e_month'] = members['etime'].dt.month
members['e_day'] = members['etime'].dt.day

In [32]:
#cleaned age and cleaned_gender
members['is_male'] = members.gender == 'male'
members['is_female'] = members.gender == 'female'
members['age_mtrue'] = members.bd.apply(lambda x: 1 if x > 10 and x < 80 else 0)
weighted_age = round(members.bd[members.age_mtrue == 1].mean())
members['age_clean'] = members.bd.apply(lambda x: x if x >10 and x < 80 else weighted_age)
#Another method to clean data is based on their registration method, but all via 16 is 0

In [98]:
songs.language.value_counts()
song_index_zip =zip(songs.languages.unique())
songs['language_cate'] = songs.apply()

 52.0    1336694
-1.0      639467
 3.0      106295
 17.0      92518
 24.0      41744
 31.0      39201
 10.0      15482
 45.0      14435
 59.0       8098
 38.0       2385
Name: language, dtype: int64

In [145]:
v = pd.isnull(song_extra_info.isrc).sum()
song_extra_info[pd.isnull(song_extra_info.isrc)]

,song_id,name,isrc
15,uzWI7xZfL3gL2/B4ptZs0XfBuGC20ydak01SjhFuEtc=,如此美麗的妳,NaN
16,EXBuTr6J7UY6MDozwT/UDRVnmW0VGRVfeGBzrxVlX3k=,老鷹之歌,NaN
22,wSdWNC+BceQC4dBIgdNX7SGVXSFBhOCYTCWFtcvjmTw=,只要我長大,NaN
27,HCLMm7t3Wz4j3Bfc68yQ/kn32NQrM3OrfZmaxJIPoT8=,誰人愛我親像你,NaN
48,5Bodh/4SCpTLJiOSS5tHPXR7ra+Nk6FmrDHiHVEQNNI=,Shining Day,NaN
49,bNNgwWLzKX1Ox8n0sOlCsGEoxq9fEqSeTyHF+r0yA4o=,擔心,NaN
51,u3d60xvx0by7NOV/UZfvuGALrC5qRIeL1U1jNcBh3FM=,鴛鴦路,NaN
56,TdoS0gWqVxxNecd+124PnSbRkJxczG0t7bZxxkAspFU=,向前衝,NaN
60,g5wxmUw0UHCN9xnC0chMZDJBgriJr6/Db8AUwof611k=,男人的汗,NaN
65,ngL3kHTPKfDzxIAXDcbLiBcAByh2XOj5mm8AKbki474=,跟往事乾杯,NaN
